In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import load, dump

# load files

In [2]:
mode = "train"

In [3]:
# train of test
df = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/{}_shifted_rounded.parquet".format(mode))

In [4]:
all_cols = [c for c in list(df.columns) if c not in ["customer_ID", "S_2", "first_occurance", "time_id", "end_year_month"]]
nan_related_features = [
    "number_of_observations",
    "type_0_nan",
    "type_1_nan",
    "type_2_nan"
]
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
num_features = [col for col in all_cols if col not in (cat_features + nan_related_features)]

# kurtosis

In [6]:
def kurtosis(x):
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.kurtosis(x)

In [7]:
kurtosis_agg_features = df.groupby("customer_ID")[num_features].agg(kurtosis)
kurtosis_agg_features.columns = [(x + "_kurtosis") for x in kurtosis_agg_features.columns]

In [8]:
kurtosis_agg_features.to_parquet("../input/{}_kurtosis_agg_features.parquet".format(mode))